## Part 1: Scrape Data and Clean Dataframe

In [1]:
#download/import dependencies
import sys
# !conda install --yes --prefix {sys.prefix} -c conda-forge geocoder
# !conda install --yes --prefix {sys.prefix} -c conda-forge folium=0.5.0 
# !conda install --yes --prefix {sys.prefix} -c conda-forge geopy

import numpy as np 
import geocoder
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import pandas as pd
from unicodedata import normalize
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



In [2]:
###download dataset

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#download tables from wikipedia
wiki = pd.read_html(url, header = 0)

print(f'Total tables: {len(wiki)}')

#assign df and check data
postcodes = wiki[0]
postcodes.head()
#postcodes.info()


Total tables: 3


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
#remove postcodes with borough listed as N/A
postcodes = postcodes[postcodes['Borough'] != 'Not assigned']

#reset index
postcodes.reset_index()

#split neighbourhoods into multiple columns
neighbourhoods = postcodes['Neighbourhood'].str.split(",", n = 7, expand = True)

#postcodes = postcodes.join(neighbourhoods)
#postcodes.drop(columns=['Neighbourhood'])
#postcodes.rename(columns={"0": "Neighbourhood_1", "1": "Neighbourhood_2", "2": "Neighbourhood_3", 
#                          "3": "Neighbourhood_4", "4": "Neighbourhood_5", "5": "Neighbourhood_6", 
#                          "6": "Neighbourhood_7", "7": "Neighbourhood_8"})

In [4]:
postcodes.shape

(103, 3)


## Part 2: Get Lat and Lon for Neighbourhoods

In [5]:
# #my attempt at using geocoder (doesn't work, geo.latlng returns none)

# # import geocoder
# import geocoder 

# # loop until you get the coordinates
# for idx, row in postcodes.iterrows():
#     geo = geocoder.google('{}, Toronto, Ontario'.format(postcodes.iloc[0,0]))
#     lat_lng_coords = geo.latlng
#     postcodes.iloc[idx,3] = lat_lng_coords[0]
#     postcodes.iloc[idx,4] = lat_lng_coords[1]

In [6]:
#import lat/long by csv instead

#import csv with lats/lons & check
coords = pd.read_csv("Geospatial_Coordinates.csv")
coords.head()

#merge datafreames
postcodes_coords = postcodes.merge(coords, how = 'left', on = ['Postal Code'])
postcodes_coords.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [7]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[43.7,-79.4], zoom_start=10)

# add markers to map
for lat, lng, label in zip(postcodes_coords['Latitude'], postcodes_coords['Longitude'], postcodes_coords['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Part 3: Explore and Cluster Neighbourhoods

In [8]:
CLIENT_ID = 'UM5EZ1MMXBBAIUR2BERSL0ECQCUYANKC3ZBKWHCWDSXAUHXO'
CLIENT_SECRET = 'OGSUKEERLGXANFE2YXDUDLTLGQASHHAA5CISRCEVX3DFOBG4'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UM5EZ1MMXBBAIUR2BERSL0ECQCUYANKC3ZBKWHCWDSXAUHXO
CLIENT_SECRET:OGSUKEERLGXANFE2YXDUDLTLGQASHHAA5CISRCEVX3DFOBG4


In [9]:
#explode neighbourhoods with multiple comma-delimited entries
neighbourhoods = postcodes_coords[['Neighbourhood','Latitude','Longitude']]

#create series from neighbourhood column split by ,
s = neighbourhoods['Neighbourhood'].str.split(',').apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1) # to line up with df's index
s.name = 'Neighbourhood' # needs a name to join

#remove old neighbourhoods column and join by index
del neighbourhoods['Neighbourhood']
neighbourhoods = neighbourhoods.join(s)

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=neighbourhoods['Neighbourhood'],
                                   latitudes=neighbourhoods['Latitude'],
                                   longitudes=neighbourhoods['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park
 Harbourfront
Lawrence Manor
 Lawrence Heights
Queen's Park
 Ontario Provincial Government
Islington Avenue
 Humber Valley Village
Malvern
 Rouge
Don Mills
Parkview Hill
 Woodbine Gardens
Garden District
 Ryerson
Glencairn
West Deane Park
 Princess Gardens
 Martin Grove
 Islington
 Cloverdale
Rouge Hill
 Port Union
 Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate
 Bloordale Gardens
 Old Burnhamthorpe
 Markland Wood
Guildwood
 Morningside
 West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
 Wilson Heights
 Downsview North
Thorncliffe Park
Richmond
 Adelaide
 King
Dufferin
 Dovercourt Village
Scarborough Village
Fairview
 Henry Farm
 Oriole
Northwood Park
 York University
East Toronto
 Broadview North (Old East York)
Harbourfront East
 Union Station
 Toronto Islands
Little Portugal
 Trinity


In [179]:
print(toronto_venues.shape)
toronto_venues.head()

(4319, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [181]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,96,96,96,96,96,96
Agincourt North,3,3,3,3,3,3
Albion Gardens,8,8,8,8,8,8
Bathurst Quay,15,15,15,15,15,15
Beaumond Heights,8,8,8,8,8,8
Bloordale Gardens,9,9,9,9,9,9
Broadview North (Old East York),4,4,4,4,4,4
Cabbagetown,46,46,46,46,46,46
Chinatown,64,64,64,64,64,64


In [182]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 270 uniques categories.


In [184]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [185]:
toronto_onehot.shape

(4319, 270)

In [187]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,0.00,0.010417,0.010417,0.000000,0.010417,0.000000,0.0,0.000000

In [188]:
toronto_grouped.shape

(199, 270)

In [189]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Adelaide----
             venue  freq
0      Coffee Shop  0.08
1             Café  0.05
2       Restaurant  0.04
3            Hotel  0.04
4  Thai Restaurant  0.03


---- Agincourt North----
           venue  freq
0     Playground  0.33
1           Park  0.33
2   Intersection  0.33
3    Yoga Studio  0.00
4  Metro Station  0.00


---- Albion Gardens----
                 venue  freq
0        Grocery Store  0.25
1             Pharmacy  0.12
2  Fried Chicken Joint  0.12
3       Sandwich Place  0.12
4           Beer Store  0.12


---- Bathurst Quay----
              venue  freq
0  Airport Terminal  0.13
1    Airport Lounge  0.13
2   Airport Service  0.13
3     Boat or Ferry  0.07
4  Sculpture Garden  0.07


---- Beaumond Heights----
                 venue  freq
0        Grocery Store  0.25
1             Pharmacy  0.12
2  Fried Chicken Joint  0.12
3       Sandwich Place  0.12
4           Beer Store  0.12


---- Bloordale Gardens----
               venue  freq
0           Pharmacy  0.11
1

4          Restaurant  0.05


---- Long Branch----
                venue  freq
0         Pizza Place  0.25
1  Athletics & Sports  0.12
2         Coffee Shop  0.12
3                 Pub  0.12
4      Sandwich Place  0.12


---- Maple Leaf Park----
                        venue  freq
0            Basketball Court  0.25
1                        Park  0.25
2                      Bakery  0.25
3  Construction & Landscaping  0.25
4                 Yoga Studio  0.00


---- Markland Wood----
               venue  freq
0           Pharmacy  0.11
1  Convenience Store  0.11
2     Shopping Plaza  0.11
3         Beer Store  0.11
4        Coffee Shop  0.11


---- Martin Grove Gardens----
            venue  freq
0     Pizza Place  0.25
1  Sandwich Place  0.25
2        Bus Line  0.25
3            Park  0.25
4   Luggage Store  0.00


---- Maryvale----
                       venue  freq
0                Auto Garage   0.2
1                 Smoke Shop   0.2
2                     Bakery   0.2
3  Middle Easte

4  Fried Chicken Joint  0.08


---- The Beaches West----
                  venue  freq
0                  Park  0.10
1  Fast Food Restaurant  0.10
2    Italian Restaurant  0.05
3             Pet Store  0.05
4           Pizza Place  0.05


---- The Junction North----
                  venue  freq
0              Bus Line  0.33
1  Caribbean Restaurant  0.33
2        Breakfast Spot  0.33
3           Yoga Studio  0.00
4    Mexican Restaurant  0.00


---- The Junction South----
                venue  freq
0  Mexican Restaurant  0.08
1                Café  0.08
2     Thai Restaurant  0.08
3                Park  0.04
4      Discount Store  0.04


---- The Queensway East----
                             venue  freq
0                   Baseball Field   0.5
1                     Home Service   0.5
2                      Yoga Studio   0.0
3                    Metro Station   0.0
4  Molecular Gastronomy Restaurant   0.0


---- The Queensway West----
            venue  freq
0   Tanning Salon  0.07
1

                 venue  freq
0                  Gym  0.12
1           Restaurant  0.08
2  Japanese Restaurant  0.08
3           Beer Store  0.08
4          Coffee Shop  0.08


----Dorset Park----
                   venue  freq
0      Indian Restaurant  0.33
1     Chinese Restaurant  0.17
2              Pet Store  0.17
3     Light Rail Station  0.17
4  Vietnamese Restaurant  0.17


----Downsview----
            venue  freq
0   Grocery Store  0.20
1            Park  0.13
2  Discount Store  0.07
3   Shopping Mall  0.07
4         Airport  0.07


----Dufferin----
      venue  freq
0  Pharmacy  0.13
1    Bakery  0.13
2      Pool  0.07
3       Bar  0.07
4      Café  0.07


----East Toronto----
               venue  freq
0       Intersection  0.50
1               Park  0.25
2  Convenience Store  0.25
3        Yoga Studio  0.00
4      Metro Station  0.00


----Eringate----
               venue  freq
0           Pharmacy  0.11
1  Convenience Store  0.11
2     Shopping Plaza  0.11
3         Beer 

                             venue  freq
0                           Garden   0.5
1                             Pool   0.5
2                      Men's Store   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Rouge Hill----
                        venue  freq
0                         Bar  0.33
1  Construction & Landscaping  0.33
2                Home Service  0.33
3                 Yoga Studio  0.00
4               Metro Station  0.00


----Runnymede----
                venue  freq
0         Coffee Shop  0.11
1                Café  0.08
2                 Pub  0.05
3  Italian Restaurant  0.05
4         Pizza Place  0.05


----Scarborough Village----
                             venue  freq
0          Health & Beauty Service   0.5
1                       Playground   0.5
2                      Yoga Studio   0.0
3                      Men's Store   0.0
4  Molecular Gastronomy Restaurant   0.0


----South Steeles----
                 venue  freq


In [190]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [206]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Hotel,Restaurant,Deli / Bodega,Gym,Thai Restaurant,Bookstore,Bakery,Pizza Place
1,Agincourt North,Park,Intersection,Playground,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
2,Albion Gardens,Grocery Store,Beer Store,Pharmacy,Pizza Place,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Discount Store,Department Store,Dessert Shop
3,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Coffee Shop,Bar,Boat or Ferry,Rental Car Location,Airport Gate,Airport Food Court
4,Beaumond Heights,Grocery Store,Beer Store,Pharmacy,Pizza Place,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Discount Store,Department Store,Dessert Shop


In [207]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 1, 3, 1, 3, 0, 3, 3, 3], dtype=int32)

In [209]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = postcodes_coords

# merge toronto_grouped with postcodes to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted, how = 'left', on=['Neighbourhood'])

toronto_merged.head() # check the last columns!

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [ ]:
# create map
map_clusters = folium.Map(location=[43.7,-79.4], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]